## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
Y = attrition_df[['Attrition','Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.head()

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,2,41,1,4,Yes,0,1,6,0,8
1,1,49,8,2,No,1,3,10,1,1
2,2,37,2,3,Yes,0,3,0,0,6
3,4,33,3,3,Yes,0,3,8,3,1
4,1,27,2,2,No,1,3,2,2,9


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, Y, test_size=0.2, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import LabelEncoder

#Categorical and Numerical Cols
categorical_cols = ['OverTime', 'Department', 'EducationField', 'JobRole', 'BusinessTravel']
numerical_cols = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
                  'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
                  'YearsSinceLastPromotion', 'NumCompaniesWorked']

X_numeric = attrition_df.copy()

#Label Encode
label_encoder = LabelEncoder()
X_numeric['OverTime'] = label_encoder.fit_transform(X_numeric['OverTime'])

#Find Columns
X_numeric = pd.get_dummies(X_numeric, columns=['Department', 'EducationField', 'JobRole', 'BusinessTravel'])

X_numeric = X_numeric[[c for c in X_numeric.columns if c in X_columns or c.startswith(('Department_', 'EducationField_', 'JobRole_', 'BusinessTravel_'))]]

print(X_numeric.dtypes)
print(X_numeric.head())

X_df = X_numeric

Age                                  int64
DistanceFromHome                     int64
Education                            int64
JobSatisfaction                      int64
NumCompaniesWorked                   int64
OverTime                             int64
StockOptionLevel                     int64
WorkLifeBalance                      int64
YearsAtCompany                       int64
YearsSinceLastPromotion              int64
Department_Human Resources            bool
Department_Research & Development     bool
Department_Sales                      bool
EducationField_Human Resources        bool
EducationField_Life Sciences          bool
EducationField_Marketing              bool
EducationField_Medical                bool
EducationField_Other                  bool
EducationField_Technical Degree       bool
JobRole_Healthcare Representative     bool
JobRole_Human Resources               bool
JobRole_Laboratory Technician         bool
JobRole_Manager                       bool
JobRole_Man

In [7]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_transformed = X_df.loc[X_train.index]
X_scaler.fit(X_train_transformed)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train_transformed)
X_test_transformed = X_df.loc[X_test.index]
X_test_scaled = X_scaler.transform(X_test_transformed)


In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder()

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Department']])
y_test_encoded = encoder.transform(y_test[['Department']])

In [9]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder()

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Attrition']])
y_test_encoded = encoder.transform(y_test[['Attrition']])

## Part 2: Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data.
number_input_features = len(X_train.columns)
number_input_features

# Create the input layer
input = layers.Input(shape=(number_input_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')(input)
shared_layer2 = layers.Dense(units=32, activation='relu')(shared_layer1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer
department_input = layers.Input(shape=(X_df.filter(like='Department').shape[1],), name='Department_Input')

#Create the hidden layer
department_hidden = layers.Dense(8, activation='relu', name='Department_Hidden')(department_input)

#Create the output layer
department_output = layers.Dense(X_df.filter(like='Department').shape[1], activation='softmax', name='Department_Output')(department_hidden)


In [12]:
# Create a branch for Attrition with a hidden layer and an output layer
attrition_input = layers.Input(shape=(X_df.filter(like='Attrition').shape[1],), name='Attrition_Input')

#Create the hidden layer
attrition_hidden = layers.Dense(8, activation='relu', name='Attrition_Hidden')(attrition_input)

#Create the output layer
attrition_output = layers.Dense(X_df.filter(like='Attrition').shape[1], activation='softmax', name='Department_Output')(department_hidden)

In [13]:
#Create the model
model = Model(inputs=[attrition_input, department_input], outputs=attrition_output, name="Attrition_Model")

#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Summarize the model
model.summary()

Model: "Attrition_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Department_Input          │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Department_Hidden (Dense) │ (None, 8)              │             32 │ Department_Input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Attrition_Input           │ (None, 0)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Department_Output (Dense) │ (None, 0)              │              0 │ Department_Hidden[0][… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 32 (128.00 B)

 Trainable params: 32 (128.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
y_train_attrition = y_train_encoded
y_train_department = pd.get_dummies(y_train['Department']).values
y_test_attrition = y_test_encoded
y_test_department = pd.get_dummies(y_test['Department']).values



In [23]:
# Evaluate the model with the testing data
model.evaluate(
    [X_test_scaled[:, :X_df.filter(like='Attrition').shape[1]], X_test_scaled[:, X_df.filter(like='Attrition').shape[1]:]], # Pass a list containing attrition and department data as inputs
    {'Department_Output': y_test_department},
    verbose=0
)

ValueError: Input 1 of layer "Attrition_Model" is incompatible with the layer: expected shape=(None, 3), found shape=(None, 31)

In [24]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {attrition_accuracy}")
print(f"Department Accuracy: {department_accuracy}")

NameError: name 'attrition_accuracy' is not defined

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Even though I am running across errors, Accuracy is not the best metric to use because there are more NO attritions than YES Attritions. The uneveness can throw the model off.
2. I used Softmax and Sigmoid because there were binary classifications and multi-classifications.
3. I need to fix my code so I can generate the right accuracy score...